In [6]:
#### LIBRARIES

#### BASIC
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import json

os.environ["JAVA_HOME"] = "/usr"
os.environ["SPARK_HOME"] = "/opt/spark"
#### SPARK
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import databricks.koalas as ks
spark = SparkSession.builder.master("local[*]").getOrCreate()

#### SETTINGS
%matplotlib inline
spark.sparkContext.setLogLevel("OFF")
ps.options.plotting.backend = 'matplotlib'

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 15:50:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/30 15:50:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
review = ks.read_json("./data/review.json", lines= True)
tip = ks.read_json("./data/tip.json", lines= True)
checkin = ks.read_json("./data/checkin.json", lines= True)

In [8]:
def transform_dates(dataframe,column,format):
    """
    This function recieves 1) a dataframe, 2) the name of a column containing timestamp values
    and 3) a date format. It returns the dataframe after transforming the column to the desired 
    format.
    
    Parameters:
    - dataframe: a Koalas dataframe
    - column: the name of the column containing timestamp values
    - format: the datetime format to which the column will be transformed
    """
    series = ks.to_datetime(dataframe[column], errors='coerce')
    mode = series.mode().iloc[0].strftime(format)
    series = series.apply(lambda x: mode if (x is pd.NaT) else x.strftime(format))
    return series

In [9]:
review['date'] = transform_dates(review, 'date', '%Y-%m-%d')

/opt/spark/python/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
/opt/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/opt/spark/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
/opt/spark/python/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [14]:
tip['date'] = transform_dates(tip, 'date', '%Y-%m-%d')

/opt/spark/python/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
/opt/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/opt/spark/python/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
/opt/spark/python/pyspark/sql/pandas/functions.py:394: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [16]:
review_small = review[review['date']>'2021-10-01']
review_small.to_json('./data/review_small.json')
review_big = review[review['date']<'2021-10-01']
review_big.to_json('./data/review_big.json')

In [19]:
tip_small = tip[tip['date']>'2021-10-01']
tip_small.to_json('./data/tip_small.json')
tip_big = tip[tip['date']<'2021-10-01']
tip_big.to_json('./data/tip_big.json')